#  1.はじめに

本ノートブックでは、jリーグに所属するプロサッカーチームであるアビスパ福岡の勝ち点（3:勝ち,1:引き分け,0:負け）を、予測するモデルの構築を行います。


# 2.データ収集

In [3]:
import time
from tqdm import tqdm
import numpy as np
import pandas as pd

game_data_list = []
year_list = list(range(1996,2022,1))

for year in tqdm(year_list):
    time.sleep(1)
    url = 'https://data.j-league.or.jp/SFMS01/search?competition_years='+str(year)+'&tv_relay_station_name='
    game_data = pd.read_html(url)[0]
    game_data_list.append(game_data)
    
game_data = pd.concat(game_data_list)

100%|██████████| 26/26 [00:46<00:00,  1.77s/it]


# 3.データ前処理

In [94]:
game_data = game_data.reset_index().drop('index',axis=1)

#欠損値の削除
game_data = game_data.dropna(how='any')

#福岡の試合を抽出
F_game_data = game_data[(game_data['ホーム']=='福岡')|(game_data['アウェイ']=='福岡')]

#大会をJ1、J2のみにする
F_game_data['大会'] = F_game_data[F_game_data['大会'].str.contains('Ｊ(１|２)')]['大会'].map(lambda x : x[:2])


#福岡がホーム＆アウェイ状況
F_game_data['H_or_A'] = F_game_data['ホーム'].map(lambda x : 'H' if x=='福岡' else 'A' )

#勝ち点の設定
def func_score(x):
    if x.H_or_A == 'H':
        if x.スコア[0] > x.スコア[-1]:
            return 3
        elif x.スコア[0] == x.スコア[-1]:
            return 1
        else:
            return 0
    else:
        if x.スコア[0] < x.スコア[-1]:
            return 3
        elif x.スコア[0] == x.スコア[-1]:
            return 1
        else:
            return 0

F_game_data['勝ち点'] = F_game_data.apply(lambda x:func_score(x),axis=1)

#試合月の設定
F_game_data['試合月'] = F_game_data['試合日'].map(lambda x : str(x)[:2]).astype(int)

#K/O時間を朝、昼、夜に設定
F_game_data = F_game_data.rename(columns={'K/O時刻':'K_O_時刻'})
F_game_data['K_O_時刻'] = F_game_data['K_O_時刻'].str[:2].astype(int)

def func_time(x):
    if x.K_O_時刻 < 16:
        return '昼'
    elif x.K_O_時刻 >= 16:
        return '夜'
    else:
        return '朝'

F_game_data['K_O_時間帯'] = F_game_data.apply(lambda x:func_time(x),axis=1)

#対戦相手の設定
F_game_data['対戦相手'] = pd.concat([F_game_data[~(F_game_data['アウェイ']=='福岡')]['アウェイ'].dropna(),F_game_data[~(F_game_data['ホーム']=='福岡')]['ホーム'].dropna()])

#NaN値の削除
F_game_data = F_game_data.dropna(how='any')

#いらない行の削除
F_game_data = F_game_data.drop(['節','試合日','K_O_時刻','ホーム','スコア','アウェイ','インターネット中継・TV放送','スタジアム'],axis=1)
F_game_data_d = pd.get_dummies(F_game_data)

/Users/tsukamotonaru/opt/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
<ipython-input-94-0109ff5682b9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  F_game_data['大会'] = F_game_data[F_game_data['大会'].str.contains('Ｊ(１|２)')]['大会'].map(lambda x : x[:2])
<ipython-input-94-0109ff5682b9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  F_g

# 4.モデルの構築

In [95]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

#train用データとtest用データに分割
x = F_game_data_d.drop('勝ち点',axis=1)
y = F_game_data_d['勝ち点']

x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y,test_size=0.5,random_state=0)

#ロジスティック回帰
model = LogisticRegression()
model.fit(x_train,y_train)

y_pred = model.predict(x_test)

# 5.モデル評価

In [148]:
#混合行列の作成
data_cm = metrics.confusion_matrix(y_pred,y_test)

def make_cm(matrix,columns): #confusion_matrixにラベルをつける関数の作成
    n = len(columns)
    act = ['正解データ_勝ち点']*n 
    pred = ['予測データ_勝ち点']*n
    
    cm_df = pd.DataFrame(matrix,columns=[act,columns],index=[pred,columns])
    return cm_df

#混合行列
data_cm2 = make_cm(data_cm,[0,1,3])

#各評価指標
print('正解率:' + str(metrics.accuracy_score(y_test,y_pred)))
print('適合率:' + str(metrics.precision_score(y_test,y_pred,average='macro')))
print('再現率:' + str(metrics.recall_score(y_test,y_pred,average='macro')))
print('f1値:' + str(metrics.f1_score(y_test,y_pred,average='macro')))

#各勝ち点になる確率の表示
proba_df = pd.DataFrame(model.predict_proba(x_test),columns=['勝ち点_0','勝ち点_1','勝ち点_3'],index=x_test.index)

#各偏回帰係数
coefs_0 = pd.Series(model.coef_[0],index=x.columns).sort_values()
coefs_1 = pd.Series(model.coef_[1],index=x.columns).sort_values()
coefs_3 = pd.Series(model.coef_[2],index=x.columns).sort_values()

data_cm2

正解率:0.49557522123893805
適合率:0.48442702219859884
再現率:0.4394033035986494
f1値:0.41059643554709685


正解データ_勝ち点         
                    0   1    3
予測データ_勝ち点 0        93  33   47
          1         8   8    2
          3        76  62  123

# 6.まとめ

・本ノートブックでは、jリーグに所属するプロサッカーチームであるアビスパ福岡の勝ち点（3:勝ち,1:引き分け,0:負け）を、  
　目的変数（試合年度、大会、ホームorアウェイ、試合月、キックオフの時間帯、対戦相手）から予測するモデルの構築を行いました。

・正解率は約50%であり精度はあまり良くないが、勝敗にはJ1かJ2ではないか、キックオフの時間帯が関係していることがわかった。